#  Run on Kubernetes

Run benchmarking on a kubernetes cluster with the given configuration.

Talks to kubernetes to create `n` amount of new `pods` with a dask worker inside of each
forming a `dask` cluster. Then, a function specified from `config` is being imported and
run with the given arguments. The tasks created by this `function` are being run on the
`dask` cluster for distributed computation.

The config dict must contain the following sections:
* run
* dask_cluster

Within the `run` section you need to specify:
* function:
    The complete python path to the function to be run.
* args:
    A dictionary containing the keyword args that will be used with the given function.

Within the `dask_cluster` section you can specify:
* workers:
    The amount of workers to use.
   
* worker_config: A dictionary with the following keys:
    * resources: A dictionary containig the following keys:
        * memory:
            The amount of RAM memory.
        * cpu:
            The amount of cpu's to use.
    * image: A docker image to be used (optional).
    * setup: A dictionary containing the following keys:
        * script: Location to bash script from the docker container to be run.
        * git_repository: A dictionary containing the following keys:
            * url: Link to the github repository to be cloned.
            * reference: A reference to the branch or commit to checkout at.
            * install: command run to install this repository.
        * pip_packages: A list of pip packages to be installed.
        * apt_packages: A list of apt packages to be installed.

In [1]:
config = {
    'run': {
        'function': 'btb_benchmark.main.run_benchmark',
        'args': {
            'iterations': 100,
            'challenge_types': 'xgboost',
            'detailed_output': True,
        }
    },
    'dask_cluster': {
        'workers': 58,
        'worker_config': {
            'resources': {
                'memory': '4G',
                'cpu': 4
            },
            'image': 'pythiac/btb_benchmark:latest',
        },
    },
}

After we created our config dictionary, we can now run `run_benchmark` with the specified config

In [ ]:
from btb_benchmark import run_benchmark

results = run_benchmark(config)